In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/football-predicion/E0.csv")

In [ ]:
data

In [ ]:
data.shape

In [ ]:
text = "FTHG FTAG HTHG HTAG HTR Div	Date Time Referee	HS	AS	HST	AST	HF	AF	HC	AC	HY	AY	HR	AR BFDH	BFDD	BFDA	BMGMH	BMGMD	BMGMA	BVH	BVD	BVA	BWH	BWD	BWA	CLH	CLD	CLA	LBH	LBD	LBA	PSH	PSD	PSA	MaxH	MaxD	MaxA	AvgH	AvgD	AvgA	BFEH	BFED	BFEA	B365>2.5	B365<2.5	P>2.5	P<2.5	Max>2.5	Max<2.5	Avg>2.5	Avg<2.5	BFE>2.5	BFE<2.5	AHh	B365AHH	B365AHA	PAHH	PAHA	MaxAHH	MaxAHA	AvgAHH	AvgAHA	BFEAHH	BFEAHA	B365CH	B365CD	B365CA	BFDCH	BFDCD	BFDCA	BMGMCH	BMGMCD	BMGMCA	BVCH	BVCD	BVCA	BWCH	BWCD	BWCA	CLCH	CLCD	CLCA	LBCH	LBCD	LBCA	PSCH	PSCD	PSCA	MaxCH	MaxCD	MaxCA	AvgCH	AvgCD	AvgCA	BFECH	BFECD	BFECA	B365C>2.5	B365C<2.5	PC>2.5	PC<2.5	MaxC>2.5	MaxC<2.5	AvgC>2.5	AvgC<2.5	BFEC>2.5	BFEC<2.5	AHCh	B365CAHH	B365CAHA	PCAHH	PCAHA	MaxCAHH	MaxCAHA	AvgCAHH	AvgCAHA	BFECAHH	BFECAHA"
cols = [f"{i}" for i in text.split()]
columns = data.columns

In [ ]:
n_data = data
for i in cols:
    if i in columns:
        n_data = n_data.drop(i, axis="columns")
n_data.head()

In [ ]:
n_data["res"] = n_data["FTR"].map({"H":0, "D":1, "A":2})
n_data

In [ ]:
t = n_data["HomeTeam"].unique()
teams = {i: val+1 for val, i in enumerate(t)}
teams

In [ ]:
n_data["HT"] = n_data["HomeTeam"].map(teams)
n_data["AT"] = n_data["AwayTeam"].map(teams)
n_data

In [ ]:
f_drop = ["FTR", "HomeTeam", "AwayTeam"]
for i in f_drop:
    n_data = n_data.drop(i, axis="columns")
n_data

In [ ]:
n_data["BP"] = np.where(n_data["B365A"] > n_data["B365H"], 0, 2)
n_data.head()

In [ ]:
Y = n_data["res"]
n_data = n_data.drop("res", axis="columns")
n_data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(n_data, Y, test_size = 0.2, random_state=42, stratify =Y)

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    eval_metric='mlogloss',
    learning_rate=0.1,
    max_depth=4,
    n_estimators=100,
    random_state=42
)

clf.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Home Win', 'Draw', 'Away Win']))
